In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# 1
# client_id = '233d9c8c8f5341a191566e48d4eef182'
# client_secret = '2d5eaeabe54f4361af55eb760f45fede'

# 2
# client_id = 'ef74e7127f3a4900b05b37827c49e05d'
# client_secret = 'fa9343f37e2b4138901dbe4622e3993c'

# 3
client_id = '30ddd71a6b7d4e88b3681b72c9d2fae1'
client_secret = '5360368ccbcf4e2b84ac7baf54d8882e'

# 4
# client_id = 'd6d6ddc616fa49a7a7da8d6347987f7b'
# client_secret = '068a570d50d5438d8e11d8c9cfeff92e'

# 5
# client_id = 'd17670512c2848ff93c0a6921c86e7b3'
# client_secret = 'c11716690a0f43dcb2df2d122e975fd3'

# 授权并初始化 Spotipy 客户端
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

在podcast的20大分类中，各找50个show（podcast）。下载它们的全部episodes

In [2]:
def get_shows_by_category(category, limit_per_category=50, show_limit=50):
    show_ids = []
    offset = 0
    while len(show_ids) < limit_per_category:
        # 如果已经找到了足够的 shows，则停止
        if len(show_ids) >= limit_per_category:
            break
        # 使用类别关键字进行搜索，限制为 show_limit，offset 用于分页
        results = sp.search(q=category, type='show', limit=show_limit, offset=offset, market = "US") # 限定在US内搜索
        show_ids.extend([show['id'] for show in results['shows']['items']])
        
        # 如果结果少于 show_limit，说明已经没有更多 shows
        if len(results['shows']['items']) < show_limit:
            break
        
        # 更新 offset 进行下一轮分页
        offset += show_limit
    
    # 限制返回 show_id 数量为 limit_per_category
    return show_ids[:limit_per_category]

# categories = ['Science', 'Technology', 'Comedy', 'Business', 'Health', 'Music', 'Sports', 'Culture', 'News', 'Fiction']
categories = ['Science', 'Technology', 'Comedy', 'Business', 'Health', 'Music', 'Sports', 'Culture', 'News', 
              'Fiction', 'Education', 'History', 'Society & Culture', 'True Crime', 'Arts', 'Lifestyle', 
              'Parenting', 'Religion & Spirituality', 'Self-Improvement', 'TV & Film']


all_show_ids = []

for category in categories:
    shows = get_shows_by_category(category)
    all_show_ids.extend(shows)
    print(f"Found {len(shows)} shows in category {category}")

# 输出找到的所有 show_id
print(f"Total show IDs found: {len(all_show_ids)}")

Found 50 shows in category Science
Found 50 shows in category Technology
Found 50 shows in category Comedy
Found 50 shows in category Business
Found 50 shows in category Health
Found 50 shows in category Music
Found 50 shows in category Sports
Found 50 shows in category Culture
Found 50 shows in category News
Found 50 shows in category Fiction
Found 50 shows in category Education
Found 50 shows in category History
Found 50 shows in category Society & Culture
Found 50 shows in category True Crime
Found 50 shows in category Arts
Found 50 shows in category Lifestyle
Found 50 shows in category Parenting
Found 50 shows in category Religion & Spirituality
Found 50 shows in category Self-Improvement
Found 50 shows in category TV & Film
Total show IDs found: 1000


In [12]:
# 获取token
import requests
import base64
# Spotify Token URL
token_url = "https://accounts.spotify.com/api/token"

# 编码 Client ID 和 Client Secret
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

# 请求头
headers = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded"
}

# 请求体
data = {
    "grant_type": "client_credentials"
}

# 发送 POST 请求
response = requests.post(token_url, headers=headers, data=data)

# 检查响应状态
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Access Token:", access_token)
else:
    print("Failed to retrieve access token:", response.json())

Access Token: BQAWw6IH3NMA7cUweXGj3K2ino5IIGan7Smpm8Lws_kH3oqnNiB1LEp81NKAa20cJewtEyGFUTFaWOqCQ-wyuy_tesinbqUxQM-kE8gUD7IBFm1eMNc


In [4]:
import csv
from tqdm import tqdm  # 导入进度条模块
import time

In [5]:
def get_episodes_from_show(access_token, show_id, market="US", limit=50):
    """
    获取指定 Show 的所有 Episode 信息。
    """
    base_url = f"https://api.spotify.com/v1/shows/{show_id}/episodes"
    headers = {"Authorization": f"Bearer {access_token}"}
    episodes = []  # 存储所有 Episode
    offset = 0

    while True:
        params = {
            "market": market,
            "limit": limit,
            "offset": offset,
        }

        # 发送请求
        response = requests.get(base_url, headers=headers, params=params)

        # 检查 404 错误
        if response.status_code == 404:
            print(f"Error 404: Resource not found for show_id: {show_id}")
            break  # 停止请求

        # 检查状态码，处理速率限制 (429)
        if response.status_code == 429:
            retry_after = response.headers.get("Retry-After")
            if retry_after is not None:
                retry_after = int(retry_after)  # 获取等待时间
            else:
                retry_after = 1  # 默认等待 1 秒

            print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)  # 等待 retry_after 秒后再继续
            continue  # 跳过当前循环，重新尝试请求

        # 处理其他非 200 响应
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            try:
                print(f"Response content: {response.json()}")
            except ValueError:
                print(f"Non-JSON response: {response.text}")
            break

        # 确保 JSON 解析安全
        try:
            data = response.json()
        except ValueError:
            print(f"Failed to parse JSON. Response: {response.text}")
            break

        items = data.get("items", [])
        episodes.extend(items)

        if len(items) < limit:
            break

        offset += limit

    return episodes


In [6]:
def save_episodes_to_csv(episodes, filename="episodes.csv"):
    """
    将 Episode 数据保存为 CSV 文件。
    
    Parameters:
        - episodes: Episode 列表，每个元素为字典
        - filename: 输出的 CSV 文件名 (默认 "episodes.csv")
    """
    fieldnames = ["show_id", "name", "description", "release_date", "duration_minutes", "id", "audio_preview_url", "explicit", "language"]
    # fieldnames = ["name", "description", "release_date", "duration_minutes", "id", "audio_preview_url", "explicit", "language"]
    
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for episode in episodes:
            if episode is None:
                continue  # 跳过无效的episode
            
            if isinstance(episode, dict):  # 确保episode是字典类型
            # Convert duration from milliseconds to minutes
                duration_minutes = episode.get("duration_ms", 0) / 60000  # milliseconds to minutes
                writer.writerow({
                    "show_id": episode.get("show_id", ""),
                    "name": episode.get("name"),
                    "description": episode.get("description"),
                    "release_date": episode.get("release_date"),
                    "duration_minutes": round(duration_minutes, 2),
                    "id": episode.get("id"),
                    "audio_preview_url": episode.get("audio_preview_url"),
                    "explicit": episode.get("explicit"),
                    "language": episode.get("language"),
            })

In [7]:
from tqdm import tqdm

def fetch_episodes_for_shows(access_token, show_ids, market='US', limit=50):
    all_episodes = []

    # 使用 tqdm 添加进度条
    for show_id in tqdm(show_ids, desc="Fetching episodes", unit="show"):
        # 请求API获取每个 show 的 episodes
        episodes = get_episodes_from_show(access_token, show_id, market, limit)

        # 检查 episodes 是否有效
        if episodes is None:
            print(f"Error: No response or invalid response for show_id {show_id}.")
            continue
        elif len(episodes) == 0:
            print(f"Warning: No episodes found for show_id: {show_id}")
            continue
        
        # 在 episodes 列表中每个 episode 中加入 show_id 信息
        for episode in episodes:
            if isinstance(episode, dict):  # 确保是字典类型
                episode["show_id"] = show_id
            else:
                print(f"Warning: Unexpected data structure for episode in show_id {show_id}.")
        
        all_episodes.extend(episodes)  # 将当前 show 的 episodes 添加到总列表
    
    return all_episodes


In [9]:
# show_ids = ['5lY4b5PGOvMuOYOjOVEcb9', '2n5yqzKnW0kp7ImBzZPqu4']  # 替换为你实际的show_ids
episodes1 = fetch_episodes_for_shows(access_token, all_show_ids[:50])

Fetching episodes:  42%|█████████████████████████▌                                   | 21/50 [01:30<01:32,  3.20s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.51s/show]


In [10]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes1, 'all_episodes1.csv')

In [11]:
len(episodes1)

14209

In [12]:
episodes2 = fetch_episodes_for_shows(access_token, all_show_ids[50:100])

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [07:27<00:00,  8.96s/show]


In [13]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes2, 'all_episodes2.csv')

In [14]:
len(episodes1) + len(episodes2)

40028

In [15]:
episodes3 = fetch_episodes_for_shows(access_token, all_show_ids[100:150])

Fetching episodes:  54%|████████████████████████████████▉                            | 27/50 [03:22<05:15, 13.71s/show]

Fetching episodes:  56%|██████████████████████████████████▏                          | 28/50 [03:37<05:09, 14.05s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [06:03<00:00,  7.27s/show]


In [16]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes3, 'all_episodes3.csv')

In [18]:
len(episodes1) + len(episodes2) + len(episodes3)

64977

In [19]:
episodes4 = fetch_episodes_for_shows(access_token, all_show_ids[150:200])

Fetching episodes:  92%|████████████████████████████████████████████████████████     | 46/50 [05:11<00:12,  3.04s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [05:42<00:00,  6.86s/show]


In [20]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes4, 'all_episodes4.csv')

In [21]:
len(episodes1) + len(episodes2) + len(episodes3) + len(episodes4)

86577

In [22]:
episodes5 = fetch_episodes_for_shows(access_token, all_show_ids[200:250])

Fetching episodes:   6%|███▋                                                          | 3/50 [01:46<36:24, 46.48s/show]

Fetching episodes:  30%|██████████████████▎                                          | 15/50 [03:36<06:46, 11.63s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [05:18<00:00,  6.37s/show]


In [23]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes5, 'all_episodes5.csv')

In [24]:
len(episodes1) + len(episodes2) + len(episodes3) + len(episodes4) + len(episodes5)

105116

换一组密钥

In [28]:
episodes6 = fetch_episodes_for_shows(access_token, all_show_ids[250:300])

Fetching episodes:   4%|██▍                                                           | 2/50 [00:01<00:52,  1.10s/show]

Fetching episodes:  10%|██████▏                                                       | 5/50 [00:04<00:44,  1.01show/s]

Fetching episodes:  48%|█████████████████████████████▎                               | 24/50 [00:13<00:19,  1.33show/s]

Fetching episodes:  78%|███████████████████████████████████████████████▌             | 39/50 [00:19<00:03,  3.66show/s]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.16show/s]


In [29]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes6, 'all_episodes6.csv')

In [30]:
len(episodes6)

2465

In [31]:
episodes7 = fetch_episodes_for_shows(access_token, all_show_ids[300:350])

Fetching episodes:   6%|███▋                                                          | 3/50 [00:26<06:10,  7.88s/show]

Fetching episodes:  60%|████████████████████████████████████▌                        | 30/50 [03:33<03:43, 11.19s/show]

Fetching episodes:  68%|█████████████████████████████████████████▍                   | 34/50 [04:22<03:06, 11.68s/show]

Fetching episodes:  76%|██████████████████████████████████████████████▎              | 38/50 [04:47<01:31,  7.60s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [05:14<00:00,  6.29s/show]


In [32]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes7, 'all_episodes7.csv')

In [33]:
len(episodes6) + len(episodes7)

46914

In [34]:
episodes8 = fetch_episodes_for_shows(access_token, all_show_ids[350:400])

Fetching episodes:  32%|███████████████████▌                                         | 16/50 [00:26<01:28,  2.59s/show]

Fetching episodes:  82%|██████████████████████████████████████████████████           | 41/50 [02:55<00:38,  4.25s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [03:20<00:00,  4.01s/show]


In [35]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes8, 'all_episodes8.csv')

In [36]:
len(episodes6) + len(episodes7) +  len(episodes8)

73907

In [37]:
episodes9 = fetch_episodes_for_shows(access_token, all_show_ids[400:450])

Fetching episodes:  20%|████████████▏                                                | 10/50 [00:51<04:30,  6.76s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [03:16<00:00,  3.93s/show]


In [38]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes9, 'all_episodes9.csv')

In [39]:
len(episodes6) + len(episodes7) + len(episodes8)+ len(episodes9)

104360

In [40]:
episodes10 = fetch_episodes_for_shows(access_token, all_show_ids[450:500])

Fetching episodes:  38%|███████████████████████▏                                     | 19/50 [00:19<01:12,  2.35s/show]

Fetching episodes:  44%|██████████████████████████▊                                  | 22/50 [00:22<00:38,  1.38s/show]

Fetching episodes:  56%|██████████████████████████████████▏                          | 28/50 [00:28<00:24,  1.10s/show]

Fetching episodes:  70%|██████████████████████████████████████████▋                  | 35/50 [00:43<00:26,  1.76s/show]

Fetching episodes:  88%|█████████████████████████████████████████████████████▋       | 44/50 [00:51<00:04,  1.41show/s]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [00:59<00:00,  1.19s/show]


In [41]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes10, 'all_episodes10.csv')

In [42]:
len(episodes6) + len(episodes7) + len(episodes8)+ len(episodes9) + len(episodes10) 

114090

换一组密钥

In [45]:
episodes11 = fetch_episodes_for_shows(access_token, all_show_ids[500:550])

Fetching episodes:   8%|████▉                                                         | 4/50 [00:22<03:16,  4.26s/show]

Fetching episodes:  56%|██████████████████████████████████▏                          | 28/50 [01:02<00:34,  1.58s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:28<00:00,  1.76s/show]


In [46]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes11, 'all_episodes11.csv')

In [47]:
len(episodes11) 

13158

In [48]:
episodes12 = fetch_episodes_for_shows(access_token, all_show_ids[550:600])

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:49<00:00,  2.20s/show]


In [49]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes12, 'all_episodes12.csv')

In [50]:
len(episodes11) + len(episodes12)

31043

In [51]:
episodes13 = fetch_episodes_for_shows(access_token, all_show_ids[600:650])

Fetching episodes:  48%|█████████████████████████████▎                               | 24/50 [00:55<00:55,  2.15s/show]

Fetching episodes:  60%|████████████████████████████████████▌                        | 30/50 [02:12<06:15, 18.78s/show]

Fetching episodes:  70%|██████████████████████████████████████████▋                  | 35/50 [02:20<01:08,  4.58s/show]

Fetching episodes:  80%|████████████████████████████████████████████████▊            | 40/50 [02:24<00:13,  1.31s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [02:36<00:00,  3.12s/show]


In [52]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes13, 'all_episodes13.csv')

In [53]:
len(episodes11) + len(episodes12) + len(episodes13)

54425

In [54]:
episodes14 = fetch_episodes_for_shows(access_token, all_show_ids[650:700])

Fetching episodes:  20%|████████████▏                                                | 10/50 [00:23<01:38,  2.46s/show]

Fetching episodes:  80%|████████████████████████████████████████████████▊            | 40/50 [01:03<00:05,  1.73show/s]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:42<00:00,  2.04s/show]


In [55]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes14, 'all_episodes14.csv')

In [56]:
len(episodes11) + len(episodes12) + len(episodes13) + len(episodes14)

70037

In [57]:
episodes15 = fetch_episodes_for_shows(access_token, all_show_ids[700:750])

Fetching episodes:  54%|████████████████████████████████▉                            | 27/50 [01:04<00:38,  1.67s/show]

Fetching episodes:  68%|█████████████████████████████████████████▍                   | 34/50 [01:41<02:38,  9.89s/show]

Fetching episodes:  74%|█████████████████████████████████████████████▏               | 37/50 [02:40<03:45, 17.38s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [03:35<00:00,  4.32s/show]


In [58]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes15, 'all_episodes15.csv')

In [59]:
len(episodes11) + len(episodes12) + len(episodes13) + len(episodes14) + len(episodes15) 

104121

换一组密钥

In [62]:
episodes16 = fetch_episodes_for_shows(access_token, all_show_ids[750:800])

Fetching episodes:  32%|███████████████████▌                                         | 16/50 [00:35<01:52,  3.31s/show]

Fetching episodes:  58%|███████████████████████████████████▍                         | 29/50 [00:48<00:20,  1.01show/s]

Fetching episodes:  62%|█████████████████████████████████████▊                       | 31/50 [00:49<00:17,  1.09show/s]

Fetching episodes:  64%|███████████████████████████████████████                      | 32/50 [00:51<00:18,  1.03s/show]

Fetching episodes:  82%|██████████████████████████████████████████████████           | 41/50 [01:05<00:12,  1.35s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:23<00:00,  1.67s/show]


In [63]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes16, 'all_episodes16.csv')

In [64]:
len(episodes16)

12950

In [65]:
episodes17 = fetch_episodes_for_shows(access_token, all_show_ids[800:850])

Fetching episodes:   6%|███▋                                                          | 3/50 [00:04<01:04,  1.38s/show]

Fetching episodes:  26%|███████████████▊                                             | 13/50 [00:19<00:57,  1.57s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:25<00:00,  1.71s/show]

In [66]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes17, 'all_episodes17.csv')

In [67]:
len(episodes16) + len(episodes17)

26175

In [68]:
episodes18 = fetch_episodes_for_shows(access_token, all_show_ids[850:900])

Fetching episodes:  50%|██████████████████████████████▌                              | 25/50 [01:13<01:50,  4.42s/show]

Fetching episodes:  58%|███████████████████████████████████▍                         | 29/50 [01:20<00:42,  2.04s/show]

Fetching episodes:  90%|██████████████████████████████████████████████████████▉      | 45/50 [01:46<00:04,  1.14show/s]

Fetching episodes:  96%|██████████████████████████████████████████████████████████▌  | 48/50 [01:49<00:02,  1.08s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [02:02<00:00,  2.45s/show]


In [69]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes18, 'all_episodes18.csv')

In [70]:
len(episodes16) + len(episodes17) + len(episodes18)

43184

In [71]:
episodes19 = fetch_episodes_for_shows(access_token, all_show_ids[900:950])

Fetching episodes:  20%|████████████▏                                                | 10/50 [00:53<01:54,  2.86s/show]

Fetching episodes:  24%|██████████████▋                                              | 12/50 [00:54<01:09,  1.84s/show]

Fetching episodes:  50%|██████████████████████████████▌                              | 25/50 [01:24<00:45,  1.84s/show]

Fetching episodes:  82%|██████████████████████████████████████████████████           | 41/50 [01:58<00:32,  3.61s/show]

Fetching episodes:  84%|███████████████████████████████████████████████████▏         | 42/50 [02:24<01:21, 10.23s/show]

Fetching episodes:  88%|█████████████████████████████████████████████████████▋       | 44/50 [03:07<01:46, 17.82s/show]

Fetching episodes:  96%|██████████████████████████████████████████████████████████▌  | 48/50 [03:12<00:10,  5.46s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [03:13<00:00,  3.88s/show]


In [72]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes19, 'all_episodes19.csv')

In [73]:
len(episodes16) + len(episodes17) + len(episodes18) + len(episodes19)

71295

In [74]:
episodes20 = fetch_episodes_for_shows(access_token, all_show_ids[950:1000])

Fetching episodes:  14%|████████▋                                                     | 7/50 [00:14<01:10,  1.64s/show]

Fetching episodes:  34%|████████████████████▋                                        | 17/50 [00:28<01:03,  1.92s/show]

Fetching episodes:  68%|█████████████████████████████████████████▍                   | 34/50 [00:59<00:30,  1.91s/show]

Fetching episodes:  74%|█████████████████████████████████████████████▏               | 37/50 [01:07<00:33,  2.56s/show]

Fetching episodes: 100%|█████████████████████████████████████████████████████████████| 50/50 [01:30<00:00,  1.82s/show]

In [75]:
# 保存所有episodes到CSV文件
save_episodes_to_csv(episodes20, 'all_episodes20.csv')

In [76]:
len(episodes16) + len(episodes17) + len(episodes18) + len(episodes19) + len(episodes20)

86126

### 合并

In [3]:
import pandas as pd
import glob

# 定义类别列表
categories = ['Science', 'Technology', 'Comedy', 'Business', 'Health', 'Music', 'Sports', 'Culture', 'News', 
              'Fiction', 'Education', 'History', 'Society & Culture', 'True Crime', 'Arts', 'Lifestyle', 
              'Parenting', 'Religion & Spirituality', 'Self-Improvement', 'TV & Film']

# 定义所有CSV文件的路径模式
file_pattern = 'all_episodes*.csv'

# 使用glob获取所有匹配的CSV文件路径
csv_files = glob.glob(file_pattern)

# 排序文件名，根据后缀数字排序
csv_files_sorted = sorted(csv_files, key=lambda x: int(x.split('all_episodes')[1].split('.csv')[0]))

# 用于存储所有处理后的DataFrame
all_episodes = []

# 读取每个文件并添加新列
for i, file in enumerate(csv_files_sorted):
    # 打印当前处理的文件名
    print(file)
    
    # 读取CSV文件
    df = pd.read_csv(file)
    
    # 添加新列 'category'，值为对应的类别
    df['category'] = categories[i]
    
    # 将处理后的DataFrame添加到列表
    all_episodes.append(df)

all_episodes1.csv
all_episodes2.csv
all_episodes3.csv
all_episodes4.csv
all_episodes5.csv
all_episodes6.csv
all_episodes7.csv
all_episodes8.csv
all_episodes9.csv
all_episodes10.csv
all_episodes11.csv
all_episodes12.csv
all_episodes13.csv
all_episodes14.csv
all_episodes15.csv
all_episodes16.csv
all_episodes17.csv
all_episodes18.csv
all_episodes19.csv
all_episodes20.csv


In [4]:
# 合并所有DataFrame
merged_episodes = pd.concat(all_episodes, ignore_index=True)

# 保存合并后的DataFrame到新的CSV文件
merged_episodes.to_csv('episodes_with_category.csv', index=False)

In [5]:
merged_episodes.head(1)

,show_id,name,description,release_date,duration_minutes,id,audio_preview_url,explicit,language,category
0,5lY4b5PGOvMuOYOjOVEcb9,How a Nuclear Lab Helped Catch a Serial Killer,It’s the 1990s at a medical center in Californ...,2024-11-28,46.72,383sunWBthuLTikfu7uRyJ,https://podz-content.spotifycdn.com/audio/clip...,False,en,Science


In [6]:
import pandas as pd

# 统计 'category' 列的计数
category_counts = merged_episodes['category'].value_counts()

# 转换为 DataFrame 格式
category_counts_df = category_counts.reset_index()
category_counts_df.columns = ['Category', 'Episode Count']

# 打印表格
category_counts_df

,Category,Episode Count
0,Sports,44441
1,Arts,34077
2,News,30452
3,Self-Improvement,28053
4,Culture,26962
5,Technology,25819
6,Comedy,24947
7,Society & Culture,23296
8,Business,21599
9,Health,18535


### 根据show id添加show name

In [17]:
# 读取 CSV 文件
file_path = 'episodes_with_category.csv'
data = pd.read_csv(file_path)

In [18]:
data.head(1)

,show_id,name,description,release_date,duration_minutes,id,audio_preview_url,explicit,language,category
0,5lY4b5PGOvMuOYOjOVEcb9,How a Nuclear Lab Helped Catch a Serial Killer,It’s the 1990s at a medical center in Californ...,2024-11-28,46.72,383sunWBthuLTikfu7uRyJ,https://podz-content.spotifycdn.com/audio/clip...,False,en,Science


In [8]:
def get_show_name(show_id, access_token):
    url = f'https://api.spotify.com/v1/shows/{show_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('name', 'Unknown')
    else:
        return 'Unknown'

In [19]:
# 获取唯一的 show_id
unique_show_ids = data['show_id'].unique()
len(unique_show_ids)

922

In [13]:
# 创建一个字典以缓存结果
show_name_cache = {show_id: get_show_name(show_id, access_token) for show_id in unique_show_ids}

In [25]:
# 用缓存填充数据
data['show_name'] = data['show_id'].map(show_name_cache)

In [26]:
# 使用布尔条件计数
unknown_count = (data['show_name'] == 'Unknown').sum()
print(f"There are {unknown_count} 'Unknown' entries in the 'show_name' column.")

There are 0 'Unknown' entries in the 'show_name' column.


In [23]:
data.head(1)

,show_id,name,description,release_date,duration_minutes,id,audio_preview_url,explicit,language,category,show_name
0,5lY4b5PGOvMuOYOjOVEcb9,How a Nuclear Lab Helped Catch a Serial Killer,It’s the 1990s at a medical center in Californ...,2024-11-28,46.72,383sunWBthuLTikfu7uRyJ,https://podz-content.spotifycdn.com/audio/clip...,False,en,Science,Science Vs


In [24]:
# 保存新的 CSV 文件
output_path = 'episodes_with_category_and_showname.csv'
data.to_csv(output_path, index=False)